In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from fa2 import ForceAtlas2
import numpy as np
import csv
import networkx as nx
import os

In [2]:
from couchsurfing import Api

In [3]:
#Denmark
#Get the cities from list (Wikipedia)
cities = pd.read_csv('data/Top50/top50cities.csv',delimiter=',')

In [4]:
cities

,Unnamed: 0,name,population,lat,lng,countryId,countryName,countryCode,fclName,geonameId
0,0,Moscow,10381222,55.75222,37.61556,2017370,Russia,RU,"city, village,...",524901
1,1,London,7556900,51.50853,-0.12574,2635167,United Kingdom,GB,"city, village,...",2643743
2,2,St Petersburg,5028000,59.93863,30.31413,2017370,Russia,RU,"city, village,...",498817
3,3,Berlin,3426354,52.52437,13.41053,2921044,Germany,DE,"city, village,...",2950159
4,4,Madrid,3255944,40.41650,-3.70256,2510769,Spain,ES,"city, village,...",3117735
5,5,Rome,2318895,41.89193,12.51133,3175395,Italy,IT,"city, village,...",3169070
6,6,Kyiv,2797553,50.45466,30.52380,690791,Ukraine,UA,"city, village,...",703448
7,7,Paris,2138551,48.85341,2.34880,3017382,France,FR,"city, village,...",2988507
8,8,Bucharest,1877155,44.43225,26.10626,798549,Romania,RO,"city, village,...",683506
9,9,Budapest,1741041,47.49801,19.03991,719819,Hungary,HU,"city, village,...",3054643


In [16]:
#Create API
api = Api('jacob2312@protonmail.com','Jacob2312')
print(api)

In [5]:
#import nodes to get id from nodes/hosts
hosts = pd.DataFrame() #creates a new dataframe that's empty
for idx,city in cities.iterrows():#.loc[cities['2017']>50000].iterrows():
    for p in range(1,14):
        if (os.path.exists('data/Top50/hosts/'+city['name']+str(p)+'.csv')):
            df = pd.read_csv('data/Top50/hosts/'+city['name']+str(p)+'.csv',index_col=False)
            hosts = pd.concat([hosts,df]).reset_index(drop=True)
print(len(hosts))
hosts=hosts.drop_duplicates()
print(len(hosts))

10586
10582


In [8]:
rev = pd.DataFrame() #creates a new dataframe that's empty
for idx,city in cities.iterrows():#.loc[cities['2017']>50000].iterrows():
    if os.path.exists('data/Top50/reviews/'+city['name']+'Rev.csv'):
        df = pd.read_csv('data/Top50/reviews/'+city['name']+'Rev.csv',index_col=False)
        rev = pd.concat([rev,df]).reset_index(drop=True)
print(len(rev))
rev=rev.drop_duplicates()
print(len(rev))

70695
70690


In [10]:
#Creating the graph G
G = nx.DiGraph()
links = []
user_id_unique=set(hosts['id'].tolist())
#G.add_nodes_from(user_id_unique)
G.add_nodes_from([(rows['id'],{'city':rows['city']}) for idx,rows in hosts.iterrows()])

G.add_edges_from([(rows['to'],rows['from'])for index,rows in rev[['to','from','relationshipType']].iterrows() if rows['relationshipType']=='host'])
G.add_edges_from([(rows['from'],rows['to'])for index,rows in rev[['to','from','relationshipType']].iterrows() if rows['relationshipType']=='surf'])


In [15]:
#Get all the nodes that have not an attribute city...
nodes_without_att=[]
for node in G.nodes(data=True):
    if not node[1]:
        nodes_without_att.append(node[0])
nodes_without_att[0]

2001999679

In [18]:
#Get profil data
#for user in nodes_without_att[0:1]:
test=api.get_profile_by_id(nodes_without_att[0])
    
#Store profil data
def createRevCityList(city): 
    with open('data/Top50/reviews/'+city+'Rev.csv','w',newline="") as f:
            writer = csv.writer(f,delimiter = ',')
            writer.writerow(["id",
                    "firstName",
                    "lastName",
                    "avatarUrl",
                    "address",
                    "isPostTrip",
                    "to",
                    "from",
                    "fromPublicName",
                    "fromPublicAddressId",
                    "fromPublicAddressDescription"
                    "fromAvatarUrl",
                    "fromStatus",
                    "fromIsVerified",
                    "fromIsDeleted",
                    "fromBlockedBy",
                    "inverseReference",
                    "response"])

In [21]:
test['publicAddress']['description']

'Setiabudi, Jakarta, Indonesia'

In [31]:
rev.loc[rev.fromPublicAddressDescriptionfromAvatarUrl=='Riga, Riga, Latvia'].sort_values(by=['id'])[['id','text']]

,id,text
21864,319532,A very small man with an unbelievably big hear...
21871,513804,All days what we spent together was amazing. I...
22215,565582,He trusted me and my friend Monta. He gave (ac...
30063,574842,Every king needs a queen! So if you are a girl...
61232,622059,Mihaly is a wonderful person. His interests ar...
22159,1280868,I spend a really fantastic time in Milano and ...
22157,1330485,Helpful host and great storyteller!
22149,1714465,Mau is sooooo cool and wanderful host!!he neve...
65622,1855646,Martin and Birgit are COOL! Me and my friend M...
65621,1911642,When I was in Berlin the very first time this ...


In [25]:
rev

,id,text,createdDate,experience,relationshipType,isPostTrip,to,from,fromPublicName,fromPublicAddressId,fromPublicAddressDescriptionfromAvatarUrl,fromStatus,fromIsVerified,fromIsDeleted,fromBlockedBy,inverseReference,response
0,27077307,Amazing chef and humble human being. Willing t...,2018-10-17T22:09:22Z,positive,host,True,2009059021,2001999679,Dhanush Anand,109610804.0,"Setiabudi, Jakarta, Indonesia",https://s3.amazonaws.com/ht-images.couchsurfin...,yes,True,False,NaN,NaN
1,27152728,Amazing boy with amazing family . He met me up...,2018-10-28T19:16:31Z,positive,surf,True,2009059021,2007724821,Bakhar Sardzhaeva,271153483.0,Dubai,https://s3.amazonaws.com/ht-images.couchsurfin...,no,False,False,NaN,NaN
2,26922175,￼\n\n \n\nWe use cookies to enhance your exper...,2018-09-27T14:41:06Z,positive,surf,True,2008295707,2008797517,Sun-hyung Kim,312790604.0,"Seoul, Seoul, Korea (Republic of)",https://s3.amazonaws.com/ht-images.couchsurfin...,maybe,False,False,NaN,NaN
3,26866500,"I was ill, when I come to Alexey's place, so I...",2018-09-20T17:32:11Z,positive,surf,True,2008295707,2003041472,Michelle Kuglerová,123277124.0,"Jihlava, Vysocina Region, Czechia",https://s3.amazonaws.com/ht-images.couchsurfin...,maybe,False,False,NaN,NaN
4,26818885,Didn't end up staying at his house because of....,2018-09-15T05:17:31Z,positive,surf,True,2008295707,4294146,Yvette Shaw,6999267.0,"Paihia, Northland, New Zealand",https://s3.amazonaws.com/images.couchsurfing.u...,no,False,False,NaN,NaN
5,26779351,С Алексеем было очень приятно и комфортно пров...,2018-09-10T12:56:37Z,positive,surf,True,2008295707,2008843836,Angelika Krasnozhonova,314663738.0,"Kirov Oblast, Russian Federation",https://s3.amazonaws.com/ht-images.couchsurfin...,hang,True,False,NaN,NaN
6,26176442,I had a great experience in Russia with Alexei...,2018-07-10T13:57:26Z,positive,surf,True,2008295707,2007359140,Diego Gabriel Cortez Fernandez,250795609.0,"Hangzhou, xiasha, Zhejiang, China",https://s3.amazonaws.com/ht-images.couchsurfin...,no,False,False,NaN,NaN
7,26164016,I bet Alexey is definitely a good person. He l...,2018-07-09T09:52:29Z,positive,surf,True,2008295707,2007634207,Heejeong Yang,265596282.0,"Seoul, Seoul, Korea, Republic of",https://s3.amazonaws.com/ht-images.couchsurfin...,no,False,False,NaN,NaN
8,26148751,My experience with Alexey was awesome. He is ...,2018-07-07T16:05:53Z,positive,surf,True,2008295707,2005150941,Siboney Godoy,176314139.0,"MD, USA",https://s3.amazonaws.com/ht-images.couchsurfin...,hang,False,False,NaN,NaN
9,26119628,Alexey is a really amazing host!! At the first...,2018-07-04T09:15:01Z,positive,surf,True,2008295707,1003812034,Emma Jimenez,10585071.0,"Pereira, Risaralda, Colombia",https://s3.amazonaws.com/images.couchsurfing.u...,maybe,False,False,NaN,NaN


In [33]:
#def get_hosts_latlong(self,latitude=10,longitude=10,page=1,perPage=100,radius=10):

def getHostFromCity(city,latitude,longitude):
    print("Getting results for ",city,latitude,longitude)
    for p in range(6,11):
        print(p)
        ##hosts = api.get_hosts(city,25,100,p)
        ##hosts = api.get_hosts(city,25,100,None,'best_match',"yes,maybe",None,p,last_login)
        hosts = api.get_hosts_latlong(latitude,longitude,p,100,25,4)
        writeCityResults2File(hosts,city,p)
        sleep(randint(2,5))
    
def writeCityResults2File(results,cityString,page):
    with open('data/Top50/hosts/'+cityString+str(page)+'.csv','w',newline="") as f:
        writer = csv.writer(f,delimiter = ',')
        # Write CSV Header, If you dont need that, remove this line
        writer.writerow(["id",
                "publicName",
                "avatarUrl",
                "isVerified",
                "status",
                "lastLogin",
                "aboutText",
                "responseRate",
                "responseTimeText",
                'responseRateText',
                "totalReferencesCount",
                "profileLink",
                "friendsCount",
                "languages",
                "city"])

        for host in results['results']:
            writer.writerow([host["id"],
                    host["publicName"],
                    host["avatarUrl"],
                    host["isVerified"],
                    host["status"],
                    host["lastLogin"],
                    host["aboutText"],
                    host["responseRate"],
                    host["responseTimeText"],
                    host["responseRateText"],
                    host["totalReferencesCount"],
                    host["profileLink"],
                    host["friendsCount"],
                    host["languages"],
                    cityString])

In [34]:
#import nodes to get id from nodes/hosts
def importHosts(index):
    hosts = pd.DataFrame() #creates a new dataframe that's empty
    city = cities.loc[index]
    for p in range(1,15):
        if (os.path.exists('data/Top50/hosts/'+city['name']+str(p)+'.csv')):
            df = pd.read_csv('data/Top50/hosts/'+city['name']+str(p)+'.csv')
            hosts = pd.concat([hosts,df]).reset_index(drop=True)
    return hosts

In [35]:
def createRevCityList(city): 
    with open('data/Top50/reviews/'+city+'Rev.csv','w',newline="") as f:
            writer = csv.writer(f,delimiter = ',')
            writer.writerow(["id",
                    "text",
                    "createdDate",
                    "experience",
                    "relationshipType",
                    "isPostTrip",
                    "to",
                    "from",
                    "fromPublicName",
                    "fromPublicAddressId",
                    "fromPublicAddressDescription"
                    "fromAvatarUrl",
                    "fromStatus",
                    "fromIsVerified",
                    "fromIsDeleted",
                    "fromBlockedBy",
                    "inverseReference",
                    "response"])

In [36]:
def appendReview(file,city):
    with open('data/Top50/reviews/'+city+'Rev.csv','a',newline="") as f:
        writer = csv.writer(f,delimiter = ',')
        for host in file['results']:
            writer.writerow([host["id"],
                host["text"],
                host["createdDate"],
                host["experience"],
                host["relationshipType"],
                host["isPostTrip"],
                host["to"]['id'],
                host["from"]['id'],
                host["from"]["publicName"],
                host["from"]["publicAddress"]["id"],
                host["from"]["publicAddress"]["description"],
                host["from"]["avatarUrl"],
                host["from"]["status"],
                host["from"]["isVerified"],
                host["from"]["isDeleted"],
                host["from"]["blockedBy"],
                host["inverseReference"],
                host["response"]])

In [37]:
def getReferenceForCity(index):
    hosts= importHosts(index=index)
    city = cities.loc[index]
    print("Getting results for ",city['name'])
    createRevCityList(city['name'])
    #exception for one user in Berlin id = 1005431874,3570008
    #startHost= hosts.index[hosts['id']==3570008]
    #startHost=startHost.tolist()[0]
    for idx,host in hosts.iterrows():
     #   if idx>startHost:
        print(host['id'],city['name'])
        test=api.get_references(uid=host['id'],type='host')
        appendReview(test,city['name'])
        test=api.get_references(uid=host['id'],type='surf')
        appendReview(test,city['name'])        

In [27]:
from random import randint
from time import sleep
#Get the the hosts
for idx,city in cities.iterrows():#.loc[cities['2017']>50000].iterrows():
    getHostFromCity(city['name'],city['lat'],city['lng'])
    sleep(randint(10,20))
    


Getting results for  Milan 45.46427 9.18951
6
7
8
9
10


In [ ]:
#Getting the references for all the hosts
for idx,city in cities.iterrows():#.loc[cities['2017']>50000].iterrows():
    if idx>10:
        getReferenceForCity(idx)
        sleep(randint(5,10))



Getting results for  Minsk
2009249857 Minsk
2008909190 Minsk
4985432 Minsk
2001673201 Minsk
1002022585 Minsk
2000086073 Minsk
1000196800 Minsk
2003593138 Minsk
3398646 Minsk
2007846013 Minsk
2003101607 Minsk
2006132806 Minsk
2323554 Minsk
1007325029 Minsk
2001622649 Minsk
1005782030 Minsk
2008030003 Minsk
2386991 Minsk
2003199763 Minsk
1000371150 Minsk
2008993587 Minsk
2001997335 Minsk
3408547 Minsk
2003325503 Minsk
2732504 Minsk
2001734239 Minsk
1002291640 Minsk
2002452034 Minsk
4720999 Minsk
5447401 Minsk
1005212343 Minsk
2005078480 Minsk
1005571872 Minsk
3261093 Minsk
4175060 Minsk
1007225657 Minsk
2006254669 Minsk
4223795 Minsk
4466358 Minsk
2006195867 Minsk
1003273944 Minsk
2008707692 Minsk
2002176110 Minsk
2002055839 Minsk
2002528744 Minsk
3591863 Minsk
2006376576 Minsk
2005256021 Minsk
2002536537 Minsk
2006650984 Minsk
2002273319 Minsk
4405508 Minsk
2009212900 Minsk
1003886528 Minsk


In [66]:
hosts

,id,publicName,avatarUrl,isVerified,status,lastLogin,aboutText,responseRate,responseTimeText,responseRateText,totalReferencesCount,profileLink,friendsCount,languages,city
0,2008808376,Roman Kliewer,https://tcdn.couchsurfing.com/849K74eKDSYUGj9a...,True,yes,about 11 hours,\nI’m not a fan of writing too much about myse...,100,replies within a day,100% response rate,4,/people/roman-kliewer,6,{'fluent': []},Hamburg
1,2008542675,Joshua Buck,https://tcdn.couchsurfing.com/Gi1fqNE3djnMGkS_...,True,maybe,about 11 hours,"Right now I'm studying Tourims-, Hotel- and Ev...",100,replies within 1-3 days,100% response rate,9,/people/joshua-buck,6,"{'fluent': [{'code': 'deu', 'name': 'German'},...",Hamburg
2,2008671271,Farhan Alfarisi,https://tcdn.couchsurfing.com/pF00-pO8Pg72owhK...,True,yes,6 minutes,OFF FROM HOSTING AND ONLY REPLY YOUR MESSAGE T...,100,replies within a day,100% response rate,25,/people/farhan-alfarisi,35,"{'fluent': [{'code': 'deu', 'name': 'German'},...",Hamburg
3,2008049491,Sime Lamerdin,https://tcdn.couchsurfing.com/1IsSZ5QMsdc1b7S1...,True,maybe,about 3 hours,"Hi, I am a german-finnish Longboarder, who rea...",81,replies within a week,81% response rate,7,/people/sime-lamerdin,7,"{'fluent': [{'code': 'deu', 'name': 'German'},...",Hamburg
4,2008867520,Nadine Hoffmann,https://tcdn.couchsurfing.com/fGei1IawaxuZV7fI...,True,maybe,about 21 hours,"Ich bin aufgeschlossen, manchmal verrückt, kan...",92,replies within a week,92% response rate,4,/people/nadine-hoffmann-4,1,"{'fluent': [{'code': 'deu', 'name': 'German'},...",Hamburg
5,2008981650,Monique Märtens,https://tcdn.couchsurfing.com/rqlMTjYx6HcgZhql...,True,maybe,about 1 hour,"I am quite busy at the moment, could host you ...",100,replies within 1-3 days,100% response rate,9,/users/2008981650,9,"{'fluent': [{'code': 'eng', 'name': 'English'}...",Hamburg
6,2004287267,Julius Oldehaver,https://tcdn.couchsurfing.com/f7INbS3IiLaPhZil...,True,yes,about 1 hour,"I really like hiking, travelling, camping, mee...",90,replies within 1-3 days,90% response rate,7,/people/julius-oldehaver,3,"{'fluent': [{'code': 'deu', 'name': 'German'},...",Hamburg
7,2855221,Youlin Peng,https://tcdn.couchsurfing.com/pEs-DmLm0ohYvy_I...,True,maybe,about 8 hours,"I was born in China, but raised in Germany. Tr...",-1,replies within a day,-1% response rate,9,/people/banang,13,"{'fluent': [{'code': 'deu', 'name': 'German'},...",Hamburg
8,2008284007,Christian Hoffmann,https://tcdn.couchsurfing.com/Dzo6cQq8-tkBAdnh...,True,yes,about 1 hour,Lustiger Ehrlichkeit,100,replies within a day,100% response rate,6,/users/2008284007,6,{'fluent': []},Hamburg
9,2008506921,Kathleen Maennel,https://tcdn.couchsurfing.com/nyaHP13r0DXiDUki...,True,maybe,about 2 hours,"I am an very active, easy-going and smily pers...",91,replies within 1-3 days,91% response rate,11,/people/kathleen-maennel,8,{'fluent': []},Hamburg


In [73]:
startHost= hosts.index[hosts['id']==2008981650]
startHost.tolist()


[5]

In [89]:
test = importHosts(index=3)

In [92]:
test.loc[test['id']==1005431874]

,id,publicName,avatarUrl,isVerified,status,lastLogin,aboutText,responseRate,responseTimeText,responseRateText,totalReferencesCount,profileLink,friendsCount,languages,city
346,1005431874,Diana Green,https://tcdn.couchsurfing.com/r6zzQeAiNPWdywuU...,True,maybe,8 days,Aloha couchsurfing crowd!\r\nI've joined couch...,53,takes longer than a week to reply,53% response rate,32,/people/diana_green,26,"{'fluent': [{'code': 'deu', 'name': 'German'},...",Berlin


In [96]:
test.loc[347]

id                                                             2009040605
publicName                                                    Eva Blanca 
avatarUrl               https://tcdn.couchsurfing.com/exQFrHZcLex40wLY...
isVerified                                                           True
status                                                              maybe
lastLogin                                                          3 days
aboutText               hi dear Couchsurfer Community \r\nI'm Eva Blan...
responseRate                                                           -1
responseTimeText                        takes longer than a week to reply
responseRateText                                        -1% response rate
totalReferencesCount                                                    1
profileLink                                             /users/2009040605
friendsCount                                                            5
languages               {'fluent': [{'